### Block 0: Hugging Face Login
This block logs you into your Hugging Face account, which is necessary to download and use certain models from the Hub.

In [ ]:
from huggingface_hub import notebook_login

print("--- Please log in to your Hugging Face account. ---")
notebook_login()

### Block 1: Installations
This block ensures all necessary Python libraries for the project are installed. This includes libraries for deep learning (`transformers`, `torch`), vector databases (`faiss-cpu`), natural language processing (`langchain`), and the user interface (`gradio`).

In [ ]:
print("--- Installing required packages... ---")
!pip install langchain langchain-community faiss-cpu transformers sentence-transformers gradio pandas torch plotly accelerate bitsandbytes -q

### Block 2: Imports & Global Setup
Here we import all the required modules and define global constants that will be used throughout the notebook. This includes file paths, model names, and classification labels.

In [ ]:
import os
import pandas as pd
import gradio as gr
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import torch
import re

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, logging, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

logging.set_verbosity_error()

DB_PATH = "conscious_memory"
JOURNAL_PATH = "journal_log.csv"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "google/flan-t5-large"
CLASSIFIER_MODEL = "facebook/bart-large-mnli"

EMOTION_LABELS = ["happy", "sad", "angry", "hopeful", "guilty", "calm", "anxious", "confused"]
VALUE_LABELS = ["honesty", "trust", "communication", "regret", "growth", "forgiveness", "courage"]

print("✅ Libraries imported and variables set.")

### Block 3: Core AI Logic (The "Brain")
These functions define the core reasoning and response generation capabilities of the AI. This includes both static (fallback) and dynamic (context-aware) response generation.

In [ ]:
def generate_static_response(emotion):
    """
    Provides a fallback response when not enough data exists to generate a dynamic one.
    """
    responses = {
        "happy": "I'm glad to hear that! 😊 Want to reflect more on what made you feel this way?",
        "sad": "I'm here with you. 😔 Want to talk more about what’s been bothering you?",
        "angry": "That sounds intense. 😤 Want to unpack why you feel this way?",
        "guilty": "Guilt can be heavy. 🧘 What could help you forgive yourself?",
        "hopeful": "That’s a beautiful mindset. 🌱 Want to explore what’s fueling that hope?",
        "confused": "It sounds like you're trying to make sense of things. 🤔 Want to clarify your thoughts together?",
        "calm": "Peaceful vibes. 🧘 Want to write about what’s helping you stay grounded?"
    }
    return responses.get(emotion.lower(), "I'm listening. Feel free to share more. 🫶")

def generate_dynamic_response(emotion, current_text, llm, db, persona="Supportive"):
    """
    Generates an advanced, personalized response by learning from past helpful feedback 
    and semantically similar entries from the vector database.
    """
    print("🧠 Generating a dynamic response...")
    
    persona_descriptions = {
        "Supportive": "a supportive, deeply empathetic, and insightful journal companion.",
        "Therapist-like": "a reflective, non-judgmental, and insightful therapist-like journal companion.",
        "Coach": "an encouraging, action-oriented, and insightful coach-like journal companion.",
        "Neutral": "a concise and factual journal companion.",
    }
    ai_persona_description = persona_descriptions.get(persona, persona_descriptions["Supportive"])

    if not os.path.exists(JOURNAL_PATH):
        return generate_static_response(emotion)

    df = pd.read_csv(JOURNAL_PATH)
    good_examples = df[(df['emotion'] == emotion) & (df['feedback'] == 'Insightful')]

    example_text = ""
    if not good_examples.empty:
        examples_to_use = good_examples.tail(5)
        for _, row in examples_to_use.iterrows():
            example_text += f"User Entry: \"{row['text']}\"\nHelpful Response: \"{row['ai_response']}\"\n\n"
    
    context_from_db = ""
    if db is not None:
        try:
            retrieved_docs = db.similarity_search(current_text, k=3)
            if retrieved_docs:
                context_from_db = "\nHere are some of your past reflections that resonate with your current thoughts:\n---\n"
                for i, doc in enumerate(retrieved_docs):
                    context_from_db += f"Entry on {doc.metadata.get('timestamp', 'N/A')}: \"{doc.page_content}\"\n"
                context_from_db += "---\n"
        except Exception as e:
            print(f"❌ ERROR retrieving from FAISS: {e}")

    prompt = f"""You are {ai_persona_description}
Your purpose is to help the user reflect deeper. Your responses must be thoughtful, unique, and directly relevant.

Here are examples of past responses the user found helpful for '{emotion}':
{example_text}
{context_from_db}

Read the user's new entry and generate one new, thoughtful response.

**Rules:**
1. Start with ONE specific, empathetic sentence that validates a core feeling from the user's entry.
2. End with ONE gentle, open-ended question to encourage deeper reflection.
3. Do NOT give advice, use the word "I", or add extra conversational filler.
4. Do NOT use generic phrases like "You're right" or "That's a good point."

**New User Entry:** "{current_text}"

**Your Response (validation sentence and question only):**
"""

    try:
        raw_response = llm.invoke(prompt)
        response_text = str(raw_response).strip()
        
        # Clean the response from any echoed prompt text
        response_text = response_text.replace("Your Response (validation sentence and question only):", "").strip()
        
        sentences = [s.strip() for s in re.split(r'[.?!]', response_text) if s.strip()]
        
        if sentences:
            validation_sentence = sentences[0]
            question_sentence = next((s for s in reversed(sentences) if s.endswith('?')), "What is on your mind most about this?")
            
            if not validation_sentence.endswith(('.', '?', '!')): validation_sentence += '.'
            if not question_sentence.endswith('?'): question_sentence += '?'
            
            clean_response = f"{validation_sentence.strip()} {question_sentence.strip()}"
        else:
            clean_response = "It sounds like you have a lot on your mind. Would you like to explore that further?"
            
        return clean_response

    except Exception as e:
        print(f"❌ ERROR during LLM call: {e}")
        return f"An error occurred while generating a response: {e}"

print("✅ Core AI logic defined.")

### Block 4: Data & Memory Functions
These helper functions handle the data persistence layer. They manage saving and loading journal entries to a CSV file, handling user feedback, calculating streaks, analyzing journal data for visualizations, and querying the journal history.

In [ ]:
def save_journal_to_csv(timestamp, text, emotion, value, ai_response):
    """Saves a record of the conversation to a CSV file."""
    new_entry = {"timestamp": timestamp, "text": text, "emotion": emotion, "value_theme": value, "ai_response": ai_response, "feedback": ""}
    df_columns = list(new_entry.keys())
    
    if os.path.exists(JOURNAL_PATH):
        df = pd.DataFrame([new_entry])
        df.to_csv(JOURNAL_PATH, mode='a', header=False, index=False)
    else:
        df = pd.DataFrame([new_entry], columns=df_columns)
        df.to_csv(JOURNAL_PATH, index=False)

def add_journal_entry(text, classifier, db, embedding, llm, persona_selection="Supportive"):
    """Main function to process and save a new journal entry."""
    if not text:
        return "", "", "", None

    emotion_result = classifier(text, EMOTION_LABELS)
    value_result = classifier(text, VALUE_LABELS)
    top_emotion, top_value = emotion_result['labels'][0], value_result['labels'][0]
    timestamp = datetime.now().isoformat()

    ai_response = generate_dynamic_response(top_emotion, text, llm, db, persona_selection)

    save_journal_to_csv(timestamp, text, top_emotion, top_value, ai_response)
    metadata = {"timestamp": timestamp, "emotion": top_emotion, "value_theme": top_value}
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    new_docs = splitter.split_documents([Document(page_content=text, metadata=metadata)])
    db.add_documents(new_docs)
    db.save_local(DB_PATH)

    print(f"✅ Entry added! Emotion: {top_emotion} | Value: {top_value}")
    return ai_response, top_emotion, top_value, timestamp

def handle_feedback(feedback_type, entry_timestamp):
    """Saves the user's feedback for a specific entry to the CSV."""
    if not entry_timestamp or not feedback_type: return "Please submit an entry and select a feedback option first."
    
    if os.path.exists(JOURNAL_PATH):
        df = pd.read_csv(JOURNAL_PATH, dtype={'timestamp': str})
        df.loc[df['timestamp'] == entry_timestamp, 'feedback'] = feedback_type
        df.to_csv(JOURNAL_PATH, index=False)
        return "Thank you for the feedback! I'm learning. 🧠"
    return "Journal file not found."

def calculate_streak():
    """Calculates the current daily journaling streak."""
    if not os.path.exists(JOURNAL_PATH): return 0, ""
    df = pd.read_csv(JOURNAL_PATH)
    if df.empty: return 0, ""

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    unique_dates = sorted(df['timestamp'].dt.date.unique())
    if not unique_dates: return 0, ""

    today = datetime.now().date()
    if today not in unique_dates and (today - timedelta(days=1)) not in unique_dates:
        return 0, "Start your streak today!"

    current_streak = 0
    check_date = today
    while check_date in unique_dates:
        current_streak += 1
        check_date -= timedelta(days=1)
        
    message = f"🔥 Day {current_streak} streak! Keep reflecting." if current_streak > 0 else "Start your streak today!"
    return current_streak, message

def analyze_journal():
    """Generates plots for the Analytics tab."""
    if not os.path.exists(JOURNAL_PATH) or pd.read_csv(JOURNAL_PATH).empty:
        empty_fig = go.Figure()
        return "No journal data to analyze yet.", empty_fig, empty_fig, empty_fig, empty_fig

    df = pd.read_csv(JOURNAL_PATH)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date

    emotion_counts = df['emotion'].value_counts().reset_index()
    fig1 = px.bar(emotion_counts, x='emotion', y='count', title='Emotion Frequency', color='emotion')

    value_counts = df['value_theme'].value_counts().reset_index()
    fig2 = px.pie(value_counts, names='value_theme', values='count', title='Core Value Themes', hole=.3)

    emotion_daily_counts = df.groupby(['date', 'emotion']).size().reset_index(name='count')
    fig3 = px.line(emotion_daily_counts, x='date', y='count', color='emotion', title='Emotion Trends Over Time', markers=True)

    value_daily_counts = df.groupby(['date', 'value_theme']).size().reset_index(name='count')
    fig4 = px.line(value_daily_counts, x='date', y='count', color='value_theme', title='Core Value Themes Over Time', markers=True)

    return "✅ Analysis Complete!", fig1, fig2, fig3, fig4

def generate_weekly_summary(llm):
    """Generates a summary of journal entries from the last 7 days."""
    if not os.path.exists(JOURNAL_PATH) or pd.read_csv(JOURNAL_PATH).empty:
        return "No journal data available for summary yet."

    df = pd.read_csv(JOURNAL_PATH)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    seven_days_ago = datetime.now() - timedelta(days=7)
    recent_entries_df = df[df['timestamp'] >= seven_days_ago]

    if recent_entries_df.empty: return "No entries in the last 7 days to summarize."

    summary_text_entries = "".join([f"Entry on {row['timestamp'].date()}: \"{row['text']}\"\n" for _, row in recent_entries_df.iterrows()])
    
    summary_prompt = f"""You are an insightful summarization agent. Based on the following journal entries from the past week, provide a concise, 2-3 sentence summary covering the overall emotional tone and any recurring themes. Conclude with a gentle thought or question.

    Entries:
    {summary_text_entries}

    Your Summary:
    """
    try:
        return str(llm.invoke(summary_prompt)).strip()
    except Exception as e:
        return f"An error occurred while generating the summary: {e}"

def ask_journal(query, llm, db):
    """Allows the user to ask questions about their journal history."""
    if not query: return "Please type a question to ask your journal."
    
    try:
        retrieved_docs = db.similarity_search(query, k=5)
        if not retrieved_docs: return "I couldn't find any relevant entries for that question."

        context = "".join([f"Entry on {doc.metadata.get('timestamp', 'N/A')}: {doc.page_content}\n" for doc in retrieved_docs])
        
        prompt = f"""You are an assistant that summarizes journal entries. Based on the following entries, answer the user's question concisely. If the information isn't present, say so.

        Entries:
        {context}

        User's Question: "{query}"

        Concise Answer:
        """
        return str(llm.invoke(prompt)).strip()

    except Exception as e:
        return f"An error occurred: {e}"

def download_journal_csv():
    """Lets the user download their complete journal as a CSV file."""
    if not os.path.exists(JOURNAL_PATH):
        pd.DataFrame(columns=["timestamp", "text", "emotion", "value_theme", "ai_response", "feedback"]).to_csv(JOURNAL_PATH, index=False)
    return JOURNAL_PATH

print("✅ Data and memory functions defined.")

### Block 5: Main Application Execution
This is the main block that initializes all the AI models and the vector database, then builds and launches the Gradio user interface. It defines wrapper functions to handle model scope within the Gradio app and connects all the UI components to their corresponding backend logic.

In [ ]:
from functools import partial
from transformers import AutoModelForSeq2SeqLM

if __name__ == "__main__":
    print("--- Initializing AI Models and Vector Database... ---")

    print("1. Loading embedding model...")
    embedding = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

    print("2. Setting up vector database (FAISS)...")
    if os.path.exists(DB_PATH):
        db = FAISS.load_local(DB_PATH, embedding, allow_dangerous_deserialization=True)
    else:
        db = FAISS.from_documents([Document(page_content="Welcome to your journal.")], embedding)
        db.save_local(DB_PATH)

    print("3. Loading classifier model...")
    classifier = pipeline("zero-shot-classification", model=CLASSIFIER_MODEL)

    print("4. Loading primary language model (LLM)...")
    quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL, quantization_config=quantization_config, device_map="auto")
    
    llm_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0.7, top_p=0.95)
    llm = HuggingFacePipeline(pipeline=llm_pipeline)

    print("✅ All models initialized successfully!")

    def add_journal_entry_wrapper(text, persona):
        return add_journal_entry(text, classifier, db, embedding, llm, persona)

    def generate_weekly_summary_wrapper():
        return generate_weekly_summary(llm)

    def ask_journal_wrapper(query):
        return ask_journal(query, llm, db)

    print("--- Building the User Interface... ---")
    with gr.Blocks(theme=gr.themes.Soft(), title="ConsciousAI Journal") as demo:
        gr.Markdown("# 🧠 ConsciousAI Journal")
        gr.Markdown("A safe space for self-reflection. Write about your day, and I'll help you understand your feelings and values.")

        last_entry_timestamp = gr.State()
        streak_display = gr.Textbox(label="Journaling Streak", interactive=False, value=calculate_streak()[1])
        ai_persona_selection = gr.Dropdown(["Supportive", "Therapist-like", "Coach", "Neutral"], label="Choose AI Persona:", value="Supportive", interactive=True)

        with gr.Tabs():
            with gr.TabItem("✍️ Journal"):
                text_input = gr.Textbox(label="What's on your mind?", lines=10, placeholder="Today I felt...")
                with gr.Row():
                    submit_btn = gr.Button("Submit Entry", variant="primary")
                    clear_btn = gr.Button("Clear")
                response_output = gr.Textbox(label="AI Companion's Response", interactive=False, lines=4)
                gr.Markdown("How did that response feel?")
                feedback_options = gr.Radio(["Insightful", "Made me think", "A bit generic", "Didn't feel right"], label="Choose what best describes the AI's response:", value=None)
                submit_feedback_btn = gr.Button("Submit Feedback", variant="secondary")
                feedback_status = gr.Textbox(label="Feedback Status", interactive=False)
                with gr.Row():
                    emotion_output = gr.Textbox(label="Detected Emotion", interactive=False)
                    value_output = gr.Textbox(label="Detected Core Value", interactive=False)
                with gr.Row():
                    download_btn = gr.Button("📥 Download Journal CSV")
                    csv_output = gr.File(label="Your journal data")

            with gr.TabItem("📊 Analytics"):
                analyze_btn = gr.Button("Analyze My Journal", variant="primary")
                analysis_status = gr.Textbox(label="Status", interactive=False)
                with gr.Row():
                    plot_emotion_freq = gr.Plot()
                    plot_value_pie = gr.Plot()
                with gr.Row():
                    plot_emotion_trend = gr.Plot(label="Emotion Trend")
                    plot_value_trend = gr.Plot(label="Core Value Trend")
                summary_btn = gr.Button("Generate Weekly Summary", variant="secondary")
                weekly_summary_output = gr.Textbox(label="Your Weekly Reflection", interactive=False, lines=5)

            with gr.TabItem("🧭 Ask Your Journal"):
                query_input = gr.Textbox(label="Your Question", lines=2, placeholder="When did I last feel calm?")
                ask_btn = gr.Button("Ask Journal", variant="primary")
                journal_answer_output = gr.Textbox(label="Journal's Answer", lines=5, interactive=False)

        submit_btn.click(fn=add_journal_entry_wrapper, inputs=[text_input, ai_persona_selection], outputs=[response_output, emotion_output, value_output, last_entry_timestamp]).then(fn=lambda: calculate_streak()[1], outputs=streak_display)
        clear_btn.click(lambda: ["", "", "", None, None, None], outputs=[text_input, response_output, emotion_output, value_output, feedback_status, feedback_options])
        download_btn.click(fn=download_journal_csv, outputs=csv_output)
        submit_feedback_btn.click(fn=handle_feedback, inputs=[feedback_options, last_entry_timestamp], outputs=feedback_status)
        analyze_btn.click(fn=analyze_journal, outputs=[analysis_status, plot_emotion_freq, plot_value_pie, plot_emotion_trend, plot_value_trend])
        summary_btn.click(fn=generate_weekly_summary_wrapper, outputs=weekly_summary_output)
        ask_btn.click(fn=ask_journal_wrapper, inputs=query_input, outputs=journal_answer_output)

    print("🚀 Launching Gradio App...")
    demo.launch(debug=True, share=True)